### INTERACTING WITH MLFLOW_CLIENT

In [3]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mydb.sqlite"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.search_experiments()

[<Experiment: artifact_location='file:///C:/Users/AIDOO K. ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1', creation_time=1703064795646, experiment_id='1', last_update_time=1703064795646, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1703064598517, experiment_id='0', last_update_time=1703064598517, lifecycle_stage='active', name='Default', tags={}>]

### REGISTERING A NEW MODEL

In [ ]:
client.create_experiment(name='my-cool-experiment')

In [8]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5, 
    order_by = ["metrics.rmse ASC"]
)

In [9]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.290959025049041}, params={'learning_rate': '0.20472169880371677',
  'max_depth': '17',
  'min_child_weight': '1.240261172',
  'objective': 'reg:linear',
  'reg_alpha': '0.28567896',
  'reg_lambda': '0.00426448',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "eb7e043a05c24fa88d057919d46c3435", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2023-12-20 '
                              '15:09:03.091130", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.11.4", "data": "model.xgb", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "xgboost": {"xgb_version": '
                              '"1.7.6", "data": "model.xgb", "model_class": '
                              '"xgboost.cor

In [11]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: eb7e043a05c24fa88d057919d46c3435, rmse: 6.2910
run id: 414327c4f9414860af276b483d00f8fd, rmse: 6.2910
run id: 8460feb4ddfe4cd38df1b846943a1a5f, rmse: 6.3185
run id: 7c800b7172894cc786cbe70043ff2ad5, rmse: 6.3314
run id: 41b9e385bd0c4ff4877ba4cf24ea6f77, rmse: 6.3328


In [14]:
runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "metrics.rmse < 6.3",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5, 
    order_by = ["metrics.rmse ASC"]
)

In [15]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: eb7e043a05c24fa88d057919d46c3435, rmse: 6.2910
run id: 414327c4f9414860af276b483d00f8fd, rmse: 6.2910


In [16]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [18]:
run_id = "7c800b7172894cc786cbe70043ff2ad5"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1703812349799, current_stage='None', description=None, last_updated_timestamp=1703812349799, name='nyc-taxi-regressor', run_id='7c800b7172894cc786cbe70043ff2ad5', run_link=None, source=('file:///C:/Users/AIDOO K. '
 'ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1/7c800b7172894cc786cbe70043ff2ad5/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [19]:
model_uri

'runs:/7c800b7172894cc786cbe70043ff2ad5/model'

### MODEL TRANSITIONING

In [20]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1703809851800, description='The NYC Taxi Predictor for Trip Duration\n', last_updated_timestamp=1703812349799, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1703809870068, current_stage='Staging', description='', last_updated_timestamp=1703810317336, name='nyc-taxi-regressor', run_id='8460feb4ddfe4cd38df1b846943a1a5f', run_link='', source=('file:///C:/Users/AIDOO K. '
  'ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1/8460feb4ddfe4cd38df1b846943a1a5f/artifacts/model'), status='READY', status_message=None, tags={'model': 'xgboost2'}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1703812349799, current_stage='None', description=None, last_updated_timestamp=1703812349799, name='nyc-taxi-regressor', run_id='7c800b7172894cc786cbe70043ff2ad5', run_link=None, source=('file:///C:/Users/AIDOO K. '
  'ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1/7c800b7172894cc786cbe70043ff2ad5/artifacts/model'), status='READY'

In [26]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

C:\Users\AIDOO K. ENOCH\AppData\Local\Temp\ipykernel_5784\2603280560.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [27]:
for version in latest_versions:
    print(f"version {version.version}, stage: {version.current_stage}")

version 1, stage: Staging
version 2, stage: Production
version 3, stage: None


In [36]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions=False
    )

C:\Users\AIDOO K. ENOCH\AppData\Local\Temp\ipykernel_5784\4248869892.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1703812349799, current_stage='Staging', description='This model version 3 was transitioned to Staging on 2023-12-29', last_updated_timestamp=1703820754611, name='nyc-taxi-regressor', run_id='7c800b7172894cc786cbe70043ff2ad5', run_link=None, source=('file:///C:/Users/AIDOO K. '
 'ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1/7c800b7172894cc786cbe70043ff2ad5/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [35]:
from datetime import datetime
date = datetime.today().date()
client.update_model_version(
    name = model_name,
    version = model_version,
    description = f"This model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1703812349799, current_stage='Staging', description='This model version 3 was transitioned to Staging on 2023-12-29', last_updated_timestamp=1703820614025, name='nyc-taxi-regressor', run_id='7c800b7172894cc786cbe70043ff2ad5', run_link=None, source=('file:///C:/Users/AIDOO K. '
 'ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1/7c800b7172894cc786cbe70043ff2ad5/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=3>

In [48]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [49]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [51]:
run_id = "eb7e043a05c24fa88d057919d46c3435"

In [52]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'C:\\Users\\AIDOO K. ENOCH\\Documents\\GitHub\\RGT\\MLOPS\\preprocessor'

In [53]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

C:\Users\AIDOO K. ENOCH\.conda\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [54]:
X_test = preprocess(df, dv)

In [55]:
target = "duration"
y_test = df[target].values

In [62]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

C:\Users\AIDOO K. ENOCH\.conda\envs\exp-tracking-env\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
C:\Users\AIDOO K. ENOCH\.conda\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [04:01:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: total: 26.7 s
Wall time: 4.69 s


{'rmse': 6.274500632325015}

In [63]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

C:\Users\AIDOO K. ENOCH\AppData\Local\Temp\ipykernel_5784\1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1703809851967, current_stage='Production', description='', last_updated_timestamp=1703822508667, name='nyc-taxi-regressor', run_id='eb7e043a05c24fa88d057919d46c3435', run_link='', source=('file:///C:/Users/AIDOO K. '
 'ENOCH/Documents/GitHub/RGT/MLOPS/mlruns/1/eb7e043a05c24fa88d057919d46c3435/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>